In [1]:
import pandas as pd
import numpy as np

from numpy.random import seed
from tensorflow import set_random_seed
import random as rn
import os
from nltk import tokenize 
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [2]:
# GOOGLE COLAB SETUP

# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
#2. Get the file
data_path          = 'drive/My Drive/Colab Notebooks/adaptHAN/AOBDL_code/data'
codes_path         = 'drive/My Drive/Colab Notebooks/adaptHAN/AOBDL_code/codes'
cv_models_path     = 'drive/My Drive/Colab Notebooks/adaptHAN/AOBDL_code/cv_models'
models_path        = 'drive/My Drive/Colab Notebooks/adaptHAN/AOBDL_code/models'

In [0]:
import sys
# the mock-0.3.1 dir contains testcase.py, testutils.py & mock.py
sys.path.append(f'{codes_path}')

In [5]:
import models

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Using TensorFlow backend.


In [0]:
#3. Read file as panda dataframe
train           = pd.read_csv(f'{data_path}/train_cleaned_no_punkt.csv') 
test_labelled   = pd.read_csv(f'{data_path}/test_labelled_cleaned_no_punkt.csv') 
test_unlabelled = pd.read_csv(f'{data_path}/test_unlabelled_cleaned_no_punkt.csv') 

train['mal']    = train[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']].sum(axis=1) >= 1  
train.drop(['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate'], axis=1, inplace=True)
train.comment_text.fillna("empty", inplace=True)

test_labelled['mal'] = test_labelled[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']].sum(axis=1) >= 1  
test_labelled.drop(['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate'], axis=1, inplace=True)
test_labelled.comment_text.fillna("empty", inplace=True)

test_unlabelled.comment_text.fillna("empty", inplace=True)

# CHANGE TRAIN AND TEST, MIX TO GET SIMILAR DISTRIBUTION
from sklearn.model_selection import train_test_split
rs=42
X_train1, X_test1, y_train1, y_test1  = train_test_split(train.drop('mal', axis=1), train.mal, stratify=train.mal, test_size=0.29, random_state=rs )
X_train2, X_test2, y_train2, y_test2  = train_test_split(test_labelled.drop('mal', axis=1), test_labelled.mal, stratify=test_labelled.mal, test_size=0.29, random_state=rs)

X = np.concatenate((X_train1.comment_text, X_train2.comment_text))
y = np.concatenate((y_train1, y_train2))

X_test = np.concatenate((X_test1.comment_text, X_test2.comment_text))
y_test = np.concatenate((y_test1, y_test2))

In [0]:
max_features = 40000
maxlen       = 400
dropout_rate = 0.25
rs           = 42
epochs       = 4
batch_size   = 256
embed_dim    = 50
rec_units    = 150


max_sen_len    = 100
max_sent_amount = 4

seed(rs)
set_random_seed(rs)
rn.seed(rs)

os.environ['PYTHONHASHSEED']=str(rs)

# LSTM

In [0]:
models.train_model(X=X, y=y, mtype = 'LSTM', cv=True, nfolds=5, epochs=4, cv_models_path=cv_models_path, train=False, rs=rs)

 fold 0


W0731 13:12:38.564609 139821179463552 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0731 13:12:38.566249 139821179463552 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0731 13:12:38.574337 139821179463552 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0731 13:12:38.592168 139821179463552 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0731 13:12:38.601481 

Fitting
31745/31745 [==============================] - 4s 130us/step
fold 0 average precision 0.886
fold 0 roc auc 0.976
 fold 1
Fitting
31745/31745 [==============================] - 2s 76us/step
fold 1 average precision 0.884
fold 1 roc auc 0.973
 fold 2
Fitting
31743/31743 [==============================] - 2s 75us/step
fold 2 average precision 0.888
fold 2 roc auc 0.973
 fold 3
Fitting
31743/31743 [==============================] - 2s 74us/step
fold 3 average precision 0.835
fold 3 roc auc 0.97
 fold 4
Fitting
31743/31743 [==============================] - 2s 74us/step
fold 4 average precision 0.806
fold 4 roc auc 0.97
PR-C 0.86
ROC AUC 0.972


In [0]:
models.train_model(X=X, y=y, X_test=X_test, y_test=y_test, mtype = 'LSTM', cv=False, nfolds=5, epochs=4, cv_models_path=models_path, train=False, rs=rs)

Fitting
64830/64830 [==============================] - 5s 72us/step
_________________________________
PR-C is 0.855
_________________________________

_________________________________
ROC AUC is 0.972
_________________________________


# GRU

In [0]:
models.train_model(X=X, y=y, mtype = 'GRU', cv=True, nfolds=5, epochs=4, cv_models_path=cv_models_path, train=False, rs=rs)

 fold 0
Fitting
31745/31745 [==============================] - 2s 70us/step
fold 0 average precision 0.887
fold 0 roc auc 0.975
 fold 1
Fitting
31745/31745 [==============================] - 2s 70us/step
fold 1 average precision 0.887
fold 1 roc auc 0.973
 fold 2
Fitting
31743/31743 [==============================] - 2s 70us/step
fold 2 average precision 0.894
fold 2 roc auc 0.977
 fold 3
Fitting
31743/31743 [==============================] - 2s 70us/step
fold 3 average precision 0.834
fold 3 roc auc 0.971
 fold 4
Fitting
31743/31743 [==============================] - 2s 70us/step
fold 4 average precision 0.799
fold 4 roc auc 0.97
PR-C 0.86
ROC AUC 0.973


In [0]:
models.train_model(X=X, y=y, X_test=X_test, y_test=y_test, mtype = 'GRU', cv=False, nfolds=5, epochs=4, cv_models_path=models_path, train=False, rs=rs)

Fitting
64830/64830 [==============================] - 4s 68us/step
_________________________________
PR-C is 0.859
_________________________________

_________________________________
ROC AUC is 0.973
_________________________________


# CNN

In [0]:
models.train_model(X=X, y=y, mtype = 'CNN', cv=True, nfolds=5, epochs=4, cv_models_path=cv_models_path, train=False, rs=rs)

 fold 0


W0731 14:21:03.141185 139751385892736 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0731 14:21:03.143211 139751385892736 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0731 14:21:03.147098 139751385892736 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0731 14:21:03.169735 139751385892736 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0731 14:21:03.178119 

Fitting
31745/31745 [==============================] - 3s 98us/step
fold 0 average precision 0.882
fold 0 roc auc 0.975
 fold 1
Fitting
31745/31745 [==============================] - 1s 38us/step
fold 1 average precision 0.881
fold 1 roc auc 0.974
 fold 2
Fitting
31743/31743 [==============================] - 1s 41us/step
fold 2 average precision 0.882
fold 2 roc auc 0.974
 fold 3
Fitting
31743/31743 [==============================] - 1s 38us/step
fold 3 average precision 0.829
fold 3 roc auc 0.97
 fold 4
Fitting
31743/31743 [==============================] - 1s 38us/step
fold 4 average precision 0.787
fold 4 roc auc 0.967
PR-C 0.852
ROC AUC 0.972


In [0]:
models.train_model(X=X, y=y, X_test=X_test, y_test=y_test, mtype = 'CNN', cv=False, nfolds=5, epochs=4, cv_models_path=models_path, train=False, rs=rs)

Fitting
64830/64830 [==============================] - 2s 37us/step
_________________________________
PR-C is 0.85
_________________________________

_________________________________
ROC AUC is 0.971
_________________________________


# BGRU

In [0]:
models.train_model(X=X, y=y, mtype = 'BGRU', cv=True, nfolds=5, epochs=4, cv_models_path=cv_models_path, train=False, rs=rs)

 fold 0
Fitting
31745/31745 [==============================] - 4s 132us/step
fold 0 average precision 0.89
fold 0 roc auc 0.977
 fold 1
Fitting
31745/31745 [==============================] - 4s 134us/step
fold 1 average precision 0.887
fold 1 roc auc 0.975
 fold 2
Fitting
31743/31743 [==============================] - 4s 134us/step
fold 2 average precision 0.892
fold 2 roc auc 0.974
 fold 3
Fitting
31743/31743 [==============================] - 4s 134us/step
fold 3 average precision 0.833
fold 3 roc auc 0.97
 fold 4
Fitting
31743/31743 [==============================] - 4s 135us/step
fold 4 average precision 0.805
fold 4 roc auc 0.971
PR-C 0.861
ROC AUC 0.973


In [0]:
models.train_model(X=X, y=y, X_test=X_test, y_test=y_test, mtype = 'BGRU', cv=False, nfolds=5, epochs=4, cv_models_path=models_path, train=False, rs=rs)

Fitting
64830/64830 [==============================] - 9s 132us/step
_________________________________
PR-C is 0.86
_________________________________

_________________________________
ROC AUC is 0.973
_________________________________


# BLSTM

In [0]:
models.train_model(X=X, y=y, mtype = 'BLSTM', cv=True, nfolds=5, epochs=4, cv_models_path=cv_models_path, train=False, rs=rs)

 fold 0
Fitting
31745/31745 [==============================] - 5s 150us/step
fold 0 average precision 0.882
fold 0 roc auc 0.971
 fold 1
Fitting
31745/31745 [==============================] - 5s 149us/step
fold 1 average precision 0.845
fold 1 roc auc 0.956
 fold 2
Fitting
31743/31743 [==============================] - 5s 148us/step
fold 2 average precision 0.889
fold 2 roc auc 0.974
 fold 3
Fitting
31743/31743 [==============================] - 5s 148us/step
fold 3 average precision 0.836
fold 3 roc auc 0.969
 fold 4
Fitting
31743/31743 [==============================] - 5s 149us/step
fold 4 average precision 0.795
fold 4 roc auc 0.969
PR-C 0.85
ROC AUC 0.968


In [0]:
models.train_model(X=X, y=y, X_test=X_test, y_test=y_test, mtype = 'BLSTM', cv=False, nfolds=5, epochs=4, cv_models_path=models_path, train=False, rs=rs)

Fitting
64830/64830 [==============================] - 10s 148us/step
_________________________________
PR-C is 0.852
_________________________________

_________________________________
ROC AUC is 0.971
_________________________________


# BGRU + Max Pooling

In [0]:
models.train_model(X=X, y=y, mtype = 'BGRU_max', cv=True, nfolds=5, epochs=4, cv_models_path=cv_models_path, train=False, rs=rs)

 fold 0
Fitting
31745/31745 [==============================] - 5s 152us/step
fold 0 average precision 0.887
fold 0 roc auc 0.976
 fold 1
Fitting
31745/31745 [==============================] - 5s 152us/step
fold 1 average precision 0.883
fold 1 roc auc 0.973
 fold 2
Fitting
31743/31743 [==============================] - 5s 151us/step
fold 2 average precision 0.889
fold 2 roc auc 0.975
 fold 3
Fitting
31743/31743 [==============================] - 5s 153us/step
fold 3 average precision 0.834
fold 3 roc auc 0.969
 fold 4
Fitting
31743/31743 [==============================] - 5s 153us/step
fold 4 average precision 0.803
fold 4 roc auc 0.97
PR-C 0.859
ROC AUC 0.973


In [0]:
models.train_model(X=X, y=y, X_test=X_test, y_test=y_test, mtype = 'BGRU_max', cv=False, nfolds=5, epochs=4, cv_models_path=models_path, train=False, rs=rs)

Fitting
64830/64830 [==============================] - 10s 152us/step
_________________________________
PR-C is 0.857
_________________________________

_________________________________
ROC AUC is 0.972
_________________________________


# BGRU + attention

In [0]:
models.train_model(X=X, y=y, mtype = 'BGRU_att', cv=True, nfolds=5, epochs=4, cv_models_path=cv_models_path, train=False, rs=rs)

 fold 0


W0731 14:38:52.924727 139895499155328 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0731 14:38:52.926394 139895499155328 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0731 14:38:52.929942 139895499155328 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0731 14:38:52.951069 139895499155328 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0731 14:38:52.958698 

Fitting
31745/31745 [==============================] - 8s 252us/step
fold 0 average precision 0.887
fold 0 roc auc 0.972
 fold 1
Fitting
31745/31745 [==============================] - 6s 193us/step
fold 1 average precision 0.878
fold 1 roc auc 0.969
 fold 2
Fitting
31743/31743 [==============================] - 6s 195us/step
fold 2 average precision 0.877
fold 2 roc auc 0.971
 fold 3
Fitting
31743/31743 [==============================] - 6s 193us/step
fold 3 average precision 0.822
fold 3 roc auc 0.968
 fold 4
Fitting
31743/31743 [==============================] - 6s 194us/step
fold 4 average precision 0.792
fold 4 roc auc 0.968
PR-C 0.851
ROC AUC 0.97


In [0]:
models.train_model(X=X, y=y, X_test=X_test, y_test=y_test, mtype = 'BGRU_att', cv=False, nfolds=5, epochs=4, cv_models_path=models_path, train=False, rs=rs)

Fitting
64830/64830 [==============================] - 12s 191us/step
_________________________________
PR-C is 0.856
_________________________________

_________________________________
ROC AUC is 0.972
_________________________________


# BGRU + Average Pooling

In [0]:
models.train_model(X=X, y=y, mtype = 'BGRU_avg', cv=True, nfolds=5, epochs=4, cv_models_path=cv_models_path, train=False, rs=rs)

 fold 0
Fitting
31745/31745 [==============================] - 5s 157us/step
fold 0 average precision 0.89
fold 0 roc auc 0.976
 fold 1
Fitting
31745/31745 [==============================] - 5s 155us/step
fold 1 average precision 0.879
fold 1 roc auc 0.971
 fold 2
Fitting
31743/31743 [==============================] - 5s 154us/step
fold 2 average precision 0.895
fold 2 roc auc 0.976
 fold 3
Fitting
31743/31743 [==============================] - 5s 155us/step
fold 3 average precision 0.836
fold 3 roc auc 0.97
 fold 4
Fitting
31743/31743 [==============================] - 5s 156us/step
fold 4 average precision 0.793
fold 4 roc auc 0.966
PR-C 0.859
ROC AUC 0.972


In [0]:
models.train_model(X=X, y=y, X_test=X_test, y_test=y_test, mtype = 'BGRU_avg', cv=False, nfolds=5, epochs=4, cv_models_path=models_path, train=False, rs=rs)

Fitting
64830/64830 [==============================] - 10s 153us/step
_________________________________
PR-C is 0.855
_________________________________

_________________________________
ROC AUC is 0.971
_________________________________


# psHAN

In [0]:
models.train_model(X=X, y=y, mtype = 'psHAN', cv=True, nfolds=5, epochs=4, cv_models_path=cv_models_path, train=True, rs=rs)

 fold 0
Fitting
Epoch 1/4
126974/126974 [==============================] - 67s 529us/step - loss: 0.2084 - acc: 0.9263
Epoch 2/4
126974/126974 [==============================] - 66s 516us/step - loss: 0.1282 - acc: 0.9513
Epoch 3/4
126974/126974 [==============================] - 66s 517us/step - loss: 0.1106 - acc: 0.9566
Epoch 4/4
31745/31745 [==============================] - 5s 169us/step
fold 0 average precision 0.879
fold 0 roc auc 0.971
 fold 1
Fitting
Epoch 1/4
126974/126974 [==============================] - 66s 523us/step - loss: 0.1982 - acc: 0.9291
Epoch 2/4
126974/126974 [==============================] - 65s 514us/step - loss: 0.1227 - acc: 0.9524
Epoch 3/4
126974/126974 [==============================] - 65s 516us/step - loss: 0.1113 - acc: 0.9558
Epoch 4/4
31745/31745 [==============================] - 5s 166us/step
fold 1 average precision 0.88
fold 1 roc auc 0.971
 fold 2
Fitting
Epoch 1/4
126976/126976 [==============================] - 67s 525us/step - loss: 0.2039 

In [0]:
models.train_model(X=X, y=y, X_test=X_test, y_test=y_test, mtype = 'psHAN', cv=False, nfolds=5, epochs=4, cv_models_path=models_path, train=True, rs=rs)

Fitting
Epoch 1/4
158719/158719 [==============================] - 83s 523us/step - loss: 0.1828 - acc: 0.9351
Epoch 2/4
158719/158719 [==============================] - 82s 517us/step - loss: 0.1160 - acc: 0.9553
Epoch 3/4
158719/158719 [==============================] - 82s 515us/step - loss: 0.1066 - acc: 0.9583
Epoch 4/4
64830/64830 [==============================] - 11s 163us/step
_________________________________
PR-C is 0.853
_________________________________

_________________________________
ROC AUC is 0.971
_________________________________


# HAN

In [0]:
#3. Read file as panda dataframe
train           = pd.read_csv(f'{data_path}/train_cleaned_sent_punkt.csv') 
test_labelled   = pd.read_csv(f'{data_path}/test_labelled_cleaned_sent_punkt.csv') 
test_unlabelled = pd.read_csv(f'{data_path}/test_unlabelled_cleaned_sent_punkt.csv') 

train['mal']    = train[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']].sum(axis=1) >= 1  
train.drop(['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate'], axis=1, inplace=True)
train.comment_text.fillna("empty", inplace=True)

test_labelled['mal'] = test_labelled[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']].sum(axis=1) >= 1  
test_labelled.drop(['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate'], axis=1, inplace=True)
test_labelled.comment_text.fillna("empty", inplace=True)

test_unlabelled.comment_text.fillna("empty", inplace=True)

# CHANGE TRAIN AND TEST, MIX TO GET SIMILAR DISTRIBUTION
from sklearn.model_selection import train_test_split
rs=42
X_train1, X_test1, y_train1, y_test1  = train_test_split(train.drop('mal', axis=1), train.mal, stratify=train.mal, test_size=0.29, random_state=rs )
X_train2, X_test2, y_train2, y_test2  = train_test_split(test_labelled.drop('mal', axis=1), test_labelled.mal, stratify=test_labelled.mal, test_size=0.29, random_state=rs)

X = np.concatenate((X_train1.comment_text, X_train2.comment_text))
y = np.concatenate((y_train1, y_train2))

X_test = np.concatenate((X_test1.comment_text, X_test2.comment_text))
y_test = np.concatenate((y_test1, y_test2))

In [7]:
models.train_model(X=X, y=y, mtype = 'HAN', cv=True, nfolds=5, epochs=4, cv_models_path=cv_models_path, train=True, rs=rs)

 fold 0
0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
0
10000
20000
30000


W0731 19:29:57.398841 140055631349632 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0731 19:29:57.434993 140055631349632 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0731 19:29:57.443256 140055631349632 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0731 19:29:57.469741 140055631349632 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0731 19:29:57.479296 

Fitting
Epoch 1/4
126974/126974 [==============================] - 73s 575us/step - loss: 0.1872 - acc: 0.9345
Epoch 2/4
126974/126974 [==============================] - 67s 531us/step - loss: 0.1263 - acc: 0.9527
Epoch 3/4
126974/126974 [==============================] - 67s 531us/step - loss: 0.1160 - acc: 0.9561
Epoch 4/4
31745/31745 [==============================] - 5s 166us/step
fold 0 average precision 0.87
fold 0 roc auc 0.965
 fold 1
0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
0
10000
20000
30000
Fitting
Epoch 1/4
126974/126974 [==============================] - 69s 540us/step - loss: 0.1878 - acc: 0.9339
Epoch 2/4
126974/126974 [==============================] - 67s 527us/step - loss: 0.1261 - acc: 0.9527
Epoch 3/4
126974/126974 [==============================] - 67s 527us/step - loss: 0.1147 - acc: 0.9568
Epoch 4/4
31745/31745 [==============================] - 5s 165us/step
fold 1 average precision 0.863
fold 1 roc auc 0.966
 fold 2
0
10000


In [8]:
models.train_model(X=X, y=y, X_test=X_test, y_test=y_test, mtype = 'HAN', cv=False, nfolds=5, epochs=4, cv_models_path=models_path, train=True, rs=rs)

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
0
10000
20000
30000
40000
50000
60000
Fitting
Epoch 1/4
158719/158719 [==============================] - 85s 538us/step - loss: 0.1740 - acc: 0.9405
Epoch 2/4
158719/158719 [==============================] - 84s 529us/step - loss: 0.1218 - acc: 0.9551
Epoch 3/4
158719/158719 [==============================] - 84s 530us/step - loss: 0.1116 - acc: 0.9578
Epoch 4/4
64830/64830 [==============================] - 11s 162us/step
_________________________________
PR-C is 0.833
_________________________________

_________________________________
ROC AUC is 0.964
_________________________________
